In [ ]:
from datetime import datetime
import decimal
import numpy as np


class DynamicBreakoutAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2010,1,1)
        self.SetEndDate(2010,6,30)
        self.SetCash(100000)
        self.numdays = 20
        self.ceiling,self.floor = 60,20
        self.buypoint, self.sellpoint= None, None
        self.longLiqPoint, self.shortLiqPoint, self.yesterdayclose= None, None, None
        #need to create a dic to store multi forex 
        self.bband = {}
        
        for ticker in ["EURUSD", "GBPUSD"]:  #can add more forex 
            symbol = self.AddForex(ticker, Resolution.Hour, Market.Oanda).Symbol
            self.bband[symbol] = self.BB(symbol, self.numdays, 2, MovingAverageType.Simple, Resolution.Daily)
        self.Schedule.On(self.DateRules.EveryDay(symbol), self.TimeRules.BeforeMarketClose(symbol,1),Action(self.SetSignal))
        self.SetBenchmark(symbol)
       
       
    def SetSignal(self):
        #change to dictionery
        for symbol, bband in self.bband.items():
           
            close = self.History(symbol, 31, Resolution.Daily)['close']
            todayvol = np.std(close[1:31])
            yesterdayvol = np.std(close[0:30])
            deltavol = (todayvol - yesterdayvol) / todayvol
            self.numdays = int(round(self.numdays * (1 + deltavol)))
    
            if self.numdays > self.ceiling:
               self.numdays = self.ceiling
            elif self.numdays < self.floor:
                self.numdays = self.floor
            
            self.high = self.History(symbol, self.numdays, Resolution.Daily)['high']
            self.low = self.History(symbol, self.numdays, Resolution.Daily)['low']      
    
            self.buypoint = max(self.high)
            self.sellpoint = min(self.low)
            historyclose = self.History(symbol, self.numdays, Resolution.Daily)['close'] 
            self.longLiqPoint = np.mean(historyclose)
            self.shortLiqPoint = np.mean(historyclose)
            self.yesterdayclose = historyclose.iloc[-1]
            
            # wait for our BollingerBand to fully initialize
            if not all([bband.IsReady for symbol, bband in self.bband.items()]): return
    
            holdings = self.Portfolio[symbol].Quantity
            
            #need to add additiona line to check to each symbol
            bb = self.bband[symbol]
        
            if self.yesterdayclose > self.Bolband.UpperBand.Current.Value and self.Portfolio[symbol].Price >= self.buypoint:
                self.SetHoldings(symbol, .1)
            elif self.yesterdayclose < self.Bolband.LowerBand.Current.Value and self.Portfolio[symbol].Price <= self.sellpoint:
                self.SetHoldings(symbol, -.1)
    
            if holdings > 0 and self.Portfolio[symbol].Price <= self.shortLiqPoint:
                self.Liquidate(symbol)
            elif holdings < 0 and self.Portfolio[symbol].Price >= self.shortLiqPoint:
                self.Liquidate(symbol)
          
            self.Log(str(self.yesterdayclose)+(" # of days ")+(str(self.numdays)))
            
        def OnData(self,data):
            return